# Create an Excel export job

You use an Excel export job to export data into a properly formatted Excel spreadsheet.

This example shows how to create an Excel export job request, submit it to the job
queue, and interact with the resulting Excel export job object returned by the server.

Information on how to create a properly formatted Excel export template is outside the scope of
this example. For information on the use of Excel for exporting data from Granta MI, see
the Granta MI documentation or consult your ACE representative.

## Connect to Granta MI

Import the ``Connection`` class and create the connection. For more information,
see the [Connect and access the job queue](0_Getting_started.ipynb) example.

In [ ]:
from ansys.grantami.jobqueue import Connection

server_url = "http://my_grantami_server/mi_servicelayer"
client = Connection(server_url).with_credentials("user_name", "password").connect()

## Create an ``ExcelExportJobRequest`` object

The first step in exporting an Excel file with the job queue is to create an
``ExcelExportJobRequest`` object. When creating this object, specify the name of the job, the
Excel export template file, the relevant database, and the records to export. You can also
specify an optional description and the scheduled execution date, if the export should be deferred
until that date and time.

You provide the records to export as ``ExportRecord`` objects. Records are defined in terms
of a *record history identity* and an optional *record version*, which are used to identify
a previous record version in version-controlled tables. To determine the record history identity,
use of the Granta MI Python Scripting Toolkit is recommended. For more information, consult your
ACE representative.

You can provide the Excel export template file as a relative or absolute path, or as a
``pathlib.Path`` objects.

In [ ]:
from ansys.grantami.jobqueue import ExcelExportJobRequest, ExportRecord

export_records = [
    ExportRecord(record_history_identity=120732),
    ExportRecord(record_history_identity=120733),
]
excel_export_request = ExcelExportJobRequest(
    name="Excel Export",
    description="An example excel export job",
    template_file="assets/export_template.xlsx",
    database_key="MI_Training",
    records=export_records,
)
excel_export_request

## Submit the job to the server
Next, submit the jobs to the server. There are two methods for submitting job
requests:

* ``create_job()``: Submit the job request to the server and immediately return an
  ``AsyncJob`` object in the *pending* state.
* ``create_job_and_wait()``: Submit the job request to the server and block until the job
   either completes or fails. Return an ``AsyncJob`` object in the *succeeded* or *failed* state.

This example uses the ``create_job_and_wait()`` method. For an example that shows
how to create and submit a job that runs asynchronously, see
[Schedule and modify jobs](4_Scheduling_and_modifying_jobs.ipynb).

In [ ]:
completed_job = client.create_job_and_wait(excel_export_request)

## Access output files
Finally, access the results of the job. Export jobs typically create log files and either a
single Excel output file or a ZIP file containing multiple Excel files and/or attachments.

Access the list of files generated by the job with the ``output_file_names`` property. This
returns a list of file names.

In [ ]:
completed_job.output_file_names

In general, an Excel export job returns two files:

- ``<job name>.log``: Log file of the import operation on the server
- A file containing the data exported from Granta MI

The contents of the exported data file depends on the job:

* If one Excel file is generated and no attachments are included in the export,
  the exported file is the Excel file generated by the export operation.
* If there are multiple Excel files generated, or if attachments are included in the export,
  the exported file is a ZIP file of all the exported files.

This cell shows how to access the content of the log file as ``bytes`` using the
``AsyncJob.get_file_content()`` method:

In [ ]:
log_file_name = next(name for name in completed_job.output_file_names if "log" in name)
log_file_content = completed_job.get_file_content(log_file_name)
log_file_string = log_file_content.decode("utf-8")
print(f"{log_file_name} (first 200 characters):")
print(f"{log_file_string[:500]}...")

This next cell shows how to download the Excel file to disk using the ``AsyncJob.download_file()`` method.

In [ ]:
output_file_name = next(name for name in completed_job.output_file_names if name.endswith("xlsx"))
output_path = f"./{output_file_name}"
completed_job.download_file(output_file_name, output_path)
print(f"{output_file_name} saved to disk")